In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# library written for this exercise providing additional functions for assignment submission, and others
import utils

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [17]:
# Load data
# The first two columns contains the exam scores and the third column
# contains the label.
data = np.loadtxt(os.path.join('Data', 'ex2data1.txt'), delimiter=',')

np.random.shuffle(data)

X, y = data[:, 0:2], data[:, 2]

# Setup the data matrix appropriately, and add ones for the intercept term

# Add intercept term to X
X = np.concatenate([np.ones((data.shape[0], 1)), X], axis=1)
X.shape

(100, 3)

In [3]:
new_X = np.zeros((X.shape[0],5))
new_X[:,0:3] = X[:,0:3]
new_X[:,3] = X[:,1]**2
new_X[:,4] = X[:,2]**3


# 60% training set 
X_train = new_X[0:60,:]
y_train = y[0:60]

# 20% cross validation set
X_cross = new_X[60:80,:]
y_cross = y[60:80]

# 20% testing set
X_test = new_X[80:100,:]
y_test = y[80:100]

In [11]:
X_train

array([[1.00000000e+00, 7.06615096e+01, 9.29271379e+01, 4.99304893e+03,
        8.02467928e+05],
       [1.00000000e+00, 5.02864961e+01, 4.98045388e+01, 2.52873169e+03,
        1.23539764e+05],
       [1.00000000e+00, 3.02867108e+01, 4.38949975e+01, 9.17284849e+02,
        8.45755998e+04],
       [1.00000000e+00, 3.25772002e+01, 9.55985476e+01, 1.06127397e+03,
        8.73682995e+05],
       [1.00000000e+00, 8.01901808e+01, 4.48216289e+01, 6.43046509e+03,
        9.00456853e+04],
       [1.00000000e+00, 8.22266616e+01, 4.27198785e+01, 6.76122387e+03,
        7.79632666e+04],
       [1.00000000e+00, 6.22710137e+01, 6.99544580e+01, 3.87767914e+03,
        3.42330967e+05],
       [1.00000000e+00, 7.50136584e+01, 3.06032632e+01, 5.62704894e+03,
        2.86617837e+04],
       [1.00000000e+00, 8.89138964e+01, 6.98037889e+01, 7.90568098e+03,
        3.40123774e+05],
       [1.00000000e+00, 4.95866772e+01, 5.98089510e+01, 2.45883856e+03,
        2.13943234e+05],
       [1.00000000e+00, 9.9315

In [5]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
        
    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    # ====================== YOUR CODE HERE ======================
    g = 1 / (1 + np.exp(-z))


    # =============================================================
    return g

In [6]:
def costFunction(theta, X, y):

    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ====================== YOUR CODE HERE ======================
    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    grad = (1 / m) * (h - y).dot(X)
    
    
    # =============================================================
    return J, grad

In [7]:
def computeCost(theta,X,y):
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0

    # ====================== YOUR CODE HERE ======================
    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    
    return J

Once you are done call your `costFunction` using two test cases for  $\theta$ by executing the next cell.

In [12]:
# set options for optimize.minimize
options= {'maxiter': 400}

# see documention for scipy's optimize.minimize  for description about
# the different parameters
# The function returns an object `OptimizeResult`
# We use truncated Newton algorithm for optimization which is 
# equivalent to MATLAB's fminunc
# See https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy



# Initialize fitting parameters
initial_theta = np.zeros(3)



res = optimize.minimize(costFunction,
                        initial_theta,
                        (X_train[:,0:3], y_train),
                        jac=True,
                        method='TNC',
                        options=options)

# the fun property of `OptimizeResult` object returns
# the value of costFunction at optimized theta
cost = res.fun

# the optimized theta is in the x property
theta = res.x

# Print theta to screen
print('Cost at theta found by optimize.minimize: {:.3f}'.format(cost))

print('theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}]'.format(*theta))


Cost at theta found by optimize.minimize: 0.153
theta:
	[-30.360, 0.227, 0.254]


In [9]:
def predict(theta, X):
    """
    Predict whether the label is 0 or 1 using learned logistic regression.
    Computes the predictions for X using a threshold at 0.5 
    (i.e., if sigmoid(theta.T*x) >= 0.5, predict 1)
    
    Parameters
    ----------
    theta : array_like
        Parameters for logistic regression. A vecotor of shape (n+1, ).
    
    X : array_like
        The data to use for computing predictions. The rows is the number 
        of points to compute predictions, and columns is the number of
        features.

    Returns
    -------
    p : array_like
        Predictions and 0 or 1 for each row in X. 
    
    Instructions
    ------------
    Complete the following code to make predictions using your learned 
    logistic regression parameters.You should set p to a vector of 0's and 1's    
    """
    m = X.shape[0] # Number of training examples

    # You need to return the following variables correctly
    p = np.zeros(m)

    # ====================== YOUR CODE HERE ======================
    p = np.round(sigmoid(X.dot(theta.T)))

    
    # ============================================================
    return p

In [30]:

# Compute accuracy on our training set
p = predict(theta, X_test[:,0:3])
print('Train Accuracy: {:.2f} %'.format(np.mean(p == y_test) * 100))


Train Accuracy: 80.00 %


array([-30.35962521,   0.22697746,   0.25353898])

In [ ]:
print(p[0:11])
print(y_test[0:11])

In [14]:
def costFunctionReg(theta, X, y, lambda_):
    
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ===================== YOUR CODE HERE ======================
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp
    # =============================================================
    return J, grad

In [15]:
def computeCostReg(theta,X,y,lambda_):
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0

    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(theta))
    
    return J

In [16]:

# Set regularization parameter lambda to 1 (you should vary this)
lambda_ = [0,0.01,0.05,0.1,0.5,1,5,10]
# set options for optimize.minimize
options= {'maxiter': 100}

for i in range(len(lambda_)):
    
    # Initialize fitting parameters
    initial_theta = np.zeros(5)
    
    res = optimize.minimize(costFunctionReg,
                            initial_theta,
                            (X_train[:,0:5], y_train, lambda_[i]),
                            jac=True,
                            method='TNC',
                            options=options)
    print('cost is equal = ',computeCostReg(res.x,X_cross[:,0:5],y_cross,lambda_[i]))


# the optimized theta is in the x property of the result





cost is equal =  0.6355786491555002
cost is equal =  0.6355840764740229
cost is equal =  0.635605883489712
cost is equal =  0.6356333215631964
cost is equal =  0.6358511810117679
cost is equal =  0.6361250430228301
cost is equal =  0.6383419016668068
cost is equal =  0.6411772833229004
